In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter,OrderedDict
import csv
from sklearn.ensemble import RandomForestClassifier
from statistics import mean
import json

In [3]:
emp = pd.read_csv('../data/hrdata_OSF.csv')
emp_list = {}
month_dict = {}
for em in emp.to_dict('records'):
    emp_list[em['name']] = {'bank':em['bank'], 'month':em['month']}

emp_list

{'B0004': {'bank': 'B', 'month': 24},
 'B0071': {'bank': 'B', 'month': 24},
 'B0078': {'bank': 'B', 'month': 19},
 'B0082': {'bank': 'B', 'month': 24},
 'B0115': {'bank': 'B', 'month': 24},
 'B0153': {'bank': 'B', 'month': 24},
 'B0182': {'bank': 'B', 'month': 24},
 'B0193': {'bank': 'B', 'month': 24},
 'B0226': {'bank': 'B', 'month': 20},
 'B0365': {'bank': 'B', 'month': 18},
 'B0446': {'bank': 'B', 'month': 24},
 'B0518': {'bank': 'B', 'month': 24},
 'B0578': {'bank': 'B', 'month': 24},
 'B0585': {'bank': 'B', 'month': 14},
 'B0587': {'bank': 'B', 'month': 24},
 'B0693': {'bank': 'B', 'month': 24},
 'B0703': {'bank': 'B', 'month': 8},
 'B0733': {'bank': 'B', 'month': 8},
 'B0786': {'bank': 'B', 'month': 5},
 'B0847': {'bank': 'B', 'month': 24},
 'B0875': {'bank': 'B', 'month': 24},
 'B0881': {'bank': 'B', 'month': 24},
 'B0886': {'bank': 'B', 'month': 11},
 'B0929': {'bank': 'B', 'month': 10},
 'B1028': {'bank': 'B', 'month': 24},
 'B1035': {'bank': 'B', 'month': 13},
 'B1115': {'ban

In [3]:
# contemporaneous

merger = 'm1'

# month 2-24
# individual month
r = []
r_dict = []
for i in range(23):
    X_train = pd.read_csv('./data/traintest/'+'m1_train_'+str(i+2)+'.csv')
    y_train = X_train['bank'];
    X_train = X_train.drop(columns=['bank','sender','month'], axis=1)
    X_test = pd.read_csv('./data/traintest/'+'m1_test_'+str(i+2)+'.csv')
    banks = X_test['sender']
    for b in banks.tolist():
        r_dict.append(b)
    X_test = X_test.drop(columns=['bank','sender','month'], axis=1)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train,y_train.values.ravel())
    x = clf.predict_proba(X_test)
    for e in x:
        r.append(e)

In [4]:
len(r_dict)

4486

In [5]:
len(r)

4486

In [6]:
i = 0

for prediction in r:
    score  = prediction
    emp = r_dict[i]
    if emp_list[emp]['bank'] == 'B':
        if 'diff' not in emp_list[emp]:
            emp_list[emp]['diff'] = [score[0]]
            emp_list[emp]['diff_L'] = [score[1]]
        else:
            B =  emp_list[emp]['diff']
            B.append(score[0])
            emp_list[emp]['diff'] = B

            L =  emp_list[emp]['diff_L']
            L.append(score[1])
            emp_list[emp]['diff_L'] = L
        
    i+=1

In [7]:
# HD
i = 5
pre_1 = pd.read_csv('./data/traintest/'+'m1_train_'+str(i-1)+'.csv')
pre_2 = pd.read_csv('./data/traintest/'+'m1_train_'+str(i-2)+'.csv')
pre_3 = pd.read_csv('./data/traintest/'+'m1_train_'+str(i-3)+'.csv')

frames = [pre_1, pre_2, pre_3]
X = pd.concat(frames)
y = X['bank']
X = X.drop(columns=['bank','sender','month'], axis=1)
clf = RandomForestClassifier(random_state=42)
clf.fit(X,y.values.ravel())

r2 = []
r2_dict = []
for i in range(19):
    test = pd.read_csv('./data/traintest/'+'m1_test_'+str(i+6)+'.csv')
    banks = test['sender']
    for b in banks.tolist():
        r2_dict.append(b)
    c = test.drop(columns=['bank','sender','month'], axis=1)
    x  = clf.predict_proba(c)
    for e in x:
        r2.append(e)

In [8]:
for r in range(len(r2)):
    score = r2[r]
    emp = r2_dict[r]
    if emp_list[emp]['bank'] == 'B':
        if 'B' not in emp_list[emp]:
            emp_list[emp]['ret'] = [score[0]]
            emp_list[emp]['ret_L'] = [score[1]]
        else:
            B =  emp_list[emp]['ret']
            B.append(score[0])
            emp_list[emp]['ret'] = B

            L =  emp_list[emp]['ret']
            L.append(score[1])
            emp_list[emp]['ret'] = L

        if 'email' not in emp_list[emp]:
            emp_list[emp]['email'] = 1
        else:
            email =  emp_list[emp]['email'] + 1
            emp_list[emp]['email'] = email

In [9]:
for emp in emp_list:
    if 'diff' in emp_list[emp]:
        diff = emp_list[emp]['diff']
        emp_list[emp]['diff'] = round(mean(diff),2)

        diff_L = emp_list[emp]['diff_L']
        emp_list[emp]['diff_L'] = round(mean(diff_L),2)
    if 'ret' in emp_list[emp]:
        B = emp_list[emp]['ret']
        emp_list[emp]['ret'] = round(mean(B),2)

        L = emp_list[emp]['ret_L']
        emp_list[emp]['ret_L'] = round(mean(L),2)

In [10]:
for emp in emp_list:
    temp = emp_list[emp]
    if 'ret' in temp and 'diff' in temp and temp['bank'] == 'B':
        if temp['ret'] <= 0.5 and temp['diff'] > 0.5:
            temp['innov'] = temp['ret']*temp['diff']
        else:
            temp['innov'] =  0


In [11]:
result = pd.DataFrame(columns=['month','sender','diff','ret','innov','n_emails'])

for emp in emp_list:
    temp = emp_list[emp]
    if 'ret' in temp and 'diff' in temp:
        d = {'month':temp['month'],'sender':emp,'diff':temp['diff'],'ret':temp['ret'],'innov':temp['innov'], 'n_emails':temp['email']}
        result = result.append(d, ignore_index = True)

/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_10351/4059774137.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_10351/4059774137.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_10351/4059774137.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_10351/4059774137.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [12]:
result

result.to_csv('output.csv', sep=',', encoding='utf-8')

# with open("output.json", "w") as outfile:
#     json.dump(result.to_dict('records'), outfile, indent = 4)